In [1]:
# Import packages
import os
import numpy as np 
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, pdist, squareform

In [2]:
def load_data(path, name):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    data = np.loadtxt(os.path.join(path, name))
    X, Y = data[:, 1:], data[:, 0]

    return(X, Y)

In [3]:
def shuffle_data(X, Y):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters:
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output:
    Updated weights and biases
    --------------------
    '''
    # Data is currently unshuffled; we should shuffle
    # each X[i] with its corresponding y[i]
    perm = np.random.permutation(max(Y.shape))
    X = X[perm, :]
    Y = Y[perm]

    return(X, Y)

In [4]:
def split_data(X, Y, train_percent):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    # Calculate no. of training examples based on user specified percentage
    # Here we use 2/3, 1/3 by default as required by the assignment
    n_train = round(train_percent*max(Y.shape))
    
    # Filter the dataframe to get training and testing rows
    X_train = X[:n_train, :]
    Y_train = Y[:n_train]
    
    # Validation set
    X_val = X[n_train:, :]
    Y_val = Y[n_train:]
    
    # Return statement
    return(X_train, X_val, Y_train, Y_val)

In [5]:
def get_polynomial_kernel(X, X_, d):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    return(np.power(np.dot(X, X.T), d))

In [6]:
def get_gaussian_kernel(X, X_, c):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    # Compute pairwise distances
    K = np.einsum('ij,ij->i',X, X)[:,None] + np.einsum('ij,ij->i',X_,X_) - 2*np.dot(X,X_.T)
    
    # Then apply parameter c
    K = np.exp(K*c)
    
    # Return statement
    return(K)

In [7]:
def get_accuracy(target, pred):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    return np.sum(target==pred)/max(target.shape)

In [8]:
def get_results(history):
    '''
    --------------------
    Get results
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    # Store results
    best_epoch = np.array(history["dev_accuracies"]).argmax()
    best_training_accuracy = history['accuracies'][best_epoch]
    best_dev_accuracy = history['dev_accuracies'][best_epoch]
    
    # Display results
    print(f"best training accuracy: {history['accuracies'][best_epoch]}")
    print(f"best dev accuracy: {history['dev_accuracies'][best_epoch]}")
    print(f"best epoch: {best_epoch}")
    
    return(best_epoch, best_training_accuracy, best_dev_accuracy)

In [9]:
def train_kernel_perceptron(X_train, Y_train, X_dev, Y_dev, epochs, lr, kernel, kernel_args, n_classes):
    '''
    --------------------
    Kernel perceptron algorithm
    --------------------
    Parameters: 
    X: Numpy array of training features (shape = 784 X n)
    y: Binary (1/-1) training label (shape = n X 1)
    --------------------
    Output: 
    w: trained weights
    b: trained biases
    y_preds: predictions 
    --------------------
    '''
    # Store a record of training and validation accuracies at each epoch
    history = {
        "train_accuracies": [],
        "val_accuracies": []
    }
    
    # Transform X according to the user specified kernel
    if kernel == 'polynomial':
        X_train = get_polynomial_kernel(X_train, X_train, **kernel_args)
    elif kernel == 'gaussian':
        X_train = get_gaussian_kernel(X_train, X_train, **kernel_args)
    
    # Initialize alpha weights
    A = np.zeros((n_classes, X_train.shape[0]))
    
    # Initialize the best accuracy to 0 and update this during training
    best_accuracy = 0
    
    # Run for a fixed number of epochs
    for epoch in range(epochs):
        
        # Print shapes as a test
        print('This is epoch number: {}'.format(epoch))
        
        # Track number of mistakes for this epoch here
        mistakes = 0

        # Do this for each example in the dataset
        for i in range(X_train.shape[0]):

            # Compute the prediction with the current weights:
            # dim(A.T) --> (10, 6199), dim(X_train[i, :]) ---> (6199, 1) ====> dim(y_hat) --> 10 X 1
            Y_hat = A @ X_train[i, :]
            
            # Compute predictions check
            Y = np.full(n_classes, -1)
            Y[int(Y_train[i])] = 1
            
            # Compute sign
            signs = np.ones(Y_hat.shape)
            signs[Y_hat <= 0] = -1
            
            # Check if the prediction is correct against the labels
            # If it is correct we don't need to make any updates: we just move to the next iteration
            # If it is not correct then we update the weights and biases in the direction of the label
            A[Y*Y_hat <= 0, i] -= (signs[Y*Y_hat <= 0]) 
            
            # Increment mistakes counter
            print(Y_hat)
            print(np.argmax(Y_hat))
            print(Y_train[i])
            print('The number of mistakes made on epoch {} is {}...'.format(mistakes, epoch))
        
    # Return statement
    return(history)

In [10]:
def run_kernel_perceptron_training(epochs, lr, 
                                   data_path = 'data', 
                                   name = 'zipcombo.dat', 
                                   kernel = 'polynomial', 
                                   d = 2, n_classes=10):
    '''
    --------------------
    Run perceptron algorithm to get a base-line
    --------------------
    Parameters: 
    X: Numpy array of training features (shape = 784 X n)
    y: Binary (1/0) training label (shape = n X 1)
    --------------------
    Output: 
    w: trained weights
    y_preds: predictions
    --------------------
    '''
    # Set the random seed for random number generator to ensure reproducibility
    np.random.seed(132089)

    # Prepare data for the perceptron
    X, Y = load_data(data_path, name)
    
    # Split the data into training and validation set 
    X_train, X_val, Y_train, Y_val = split_data(X, Y, 0.66666)
    
    # Construct kernel arguments dictionary
    if kernel == 'polynomial': kernel_args = {'d': d}

    # Call the perceptron training with the given epochs
    history = train_kernel_perceptron(X_train, Y_train, X_val, Y_val, epochs, lr, kernel, kernel_args, n_classes)
    
    # Get results from history
    best_epoch, best_training_accuracy, best_dev_accuracy = get_results(history)
    
    # Return statement
    return(best_epoch, best_training_accuracy, best_dev_accuracy, history)

In [11]:
def main(epochs = 10, lr = 1):
    '''
    --------------------
    Main training loop
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    # Call training function
    best_epoch, best_accuracy, best_loss, history = run_kernel_perceptron_training(epochs, lr)

In [12]:
main()

This is epoch number: 0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0
6.0
The number of mistakes made on epoch 0 is 0...
[4946.86705463 4946.86705463 4946.86705463 4946.86705463 4946.86705463
 4946.86705463 4946.86705463 4946.86705463 4946.86705463 4946.86705463]
0
5.0
The number of mistakes made on epoch 0 is 0...
[7233.07181576 7233.07181576 7233.07181576 7233.07181576 7233.07181576
 7420.81905872 7233.07181576 7233.07181576 7233.07181576 7233.07181576]
5
4.0
The number of mistakes made on epoch 0 is 0...
[-9819.22244857 -9819.22244857 -9819.22244857 -9819.22244857
  1298.80345963 -8294.00667339 -9819.22244857 -9819.22244857
 -9819.22244857 -9819.22244857]
4
7.0
The number of mistakes made on epoch 0 is 0...
[ -9402.84979227  -9402.84979227  -9402.84979227  -9402.84979227
 -13535.52033791   1686.17634448  -9402.84979227  -1521.91947692
  -9402.84979227  -9402.84979227]
5
3.0
The number of mistakes made on epoch 0 is 0...
[ -8411.87563269  -8411.87563269  -8411.87563269    -34.19807133
  -4203.81

[-22707.56144545 -60401.25744764  -4979.70006943 -28871.01113127
 -35287.68460562 -30665.90156699 -23868.84303981  -2112.99087841
  -3487.06008954 -35683.48268884]
7
2.0
The number of mistakes made on epoch 0 is 0...
[-40590.0312831  -37195.23683339   -135.22375588 -24541.36949414
 -43823.00069672 -39093.50461664 -30965.8752309   32124.59482158
 -25809.77353248 -32650.57373983]
7
7.0
The number of mistakes made on epoch 0 is 0...
[-69253.03411555  37917.80369529   1548.70235085 -66201.55947005
 -19130.70056625 -57166.00317128 -36338.10133447 -43238.4349182
 -21617.85519119 -19808.22765041]
1
1.0
The number of mistakes made on epoch 0 is 0...
[-71621.99261929  38453.70622097 -52415.66053907 -65782.4140832
 -15472.09710603 -57956.17408454 -40626.52431358 -47142.37469243
 -27043.47738922 -20045.72105353]
1
1.0
The number of mistakes made on epoch 0 is 0...
[-17857.63159132 -41045.66000398 -22349.20467687 -29605.11635188
 -32743.64596248 -41498.38335579 -13603.5182562  -30702.1487599
 -130

The number of mistakes made on epoch 0 is 0...
[-61806.99537146  30772.525441   -55357.54590545 -32323.42683772
 -26891.91905526 -21267.74604253 -38266.97132039 -31231.76751917
 -55682.73682178 -55100.63834665]
1
1.0
The number of mistakes made on epoch 0 is 0...
[-17717.40667407 -28004.17566575 -33578.51537151 -12873.44151036
  10560.8931582  -19104.25001573 -22280.26387192 -13463.19198419
 -12728.0218743    2291.58839446]
4
4.0
The number of mistakes made on epoch 0 is 0...
[-28174.80689525 -16188.39228589 -27329.9159656   -2254.60161277
  -8135.96714366  -7114.80167759  -4081.67406544  -5716.54538399
 -41860.79066063 -28874.5992063 ]
3
6.0
The number of mistakes made on epoch 0 is 0...
[-19805.40305087 -12327.06404622  -9797.41842442  13728.83365299
 -12824.22116994  -3876.65843559 -30104.11096735 -12740.5279546
 -28796.41956348 -29102.26896285]
3
5.0
The number of mistakes made on epoch 0 is 0...
[-36880.45655612 -36314.53865919 -38121.14273998 -20884.10955322
 -20956.45194125  174

 -19325.23711193 -21535.35748383]
5
8.0
The number of mistakes made on epoch 0 is 0...
[-35181.11381776 -10487.1988178   10383.3566975  -28737.30652409
 -48050.77536366 -17530.68870915 -33851.21096828 -19440.51706209
 -38157.45572932 -28321.04377655]
2
2.0
The number of mistakes made on epoch 0 is 0...
[-66059.57469489  41976.50916025 -62953.88752484 -52903.08417016
 -36674.30618309 -47307.70645882 -45512.01192123 -41517.80625626
 -63654.41291437 -33050.15111288]
1
1.0
The number of mistakes made on epoch 0 is 0...
[ -7677.64983812 -36046.24920719   1929.70049071 -17475.96819243
 -12521.61353167  -5789.02249956  19979.13275693 -10150.08577181
 -25972.27384331 -15606.68746682]
6
6.0
The number of mistakes made on epoch 0 is 0...
[-52321.58354581  14620.67480454 -63097.93999621 -42627.0039362
 -35661.03953232 -31792.43391498 -37413.17384538 -24227.19037303
 -60309.03803282 -41721.62764214]
1
1.0
The number of mistakes made on epoch 0 is 0...
[-14447.65889735 -54869.67558167 -30172.419478

 -40612.5519362  -32271.09905711]
3
3.0
The number of mistakes made on epoch 0 is 0...
[-23737.25921868 -48010.49541835  26339.59703967 -26106.94978579
 -44642.50599903 -44769.8468397  -19035.13037838 -31106.20492362
 -20153.61026268 -28406.18126665]
2
2.0
The number of mistakes made on epoch 0 is 0...
[-23552.89193924 -13536.5425924  -46125.70213735 -32520.33766774
 -41861.43281946 -41157.57755604 -51390.33980208 -32680.84649212
  14184.08737183 -35610.34592766]
8
8.0
The number of mistakes made on epoch 0 is 0...
[-63271.55256827  33592.48473412 -70367.78173843 -47923.93073661
 -23989.97200068 -58648.0259364  -33439.00939094 -56473.49043452
 -40410.23955104 -44125.77998206]
1
1.0
The number of mistakes made on epoch 0 is 0...
[-66925.87336273  38215.8831025  -72136.40528723 -49254.60638884
 -22146.70874788 -58456.9326275  -38158.78256051 -56751.72594814
 -51045.73197463 -49952.90569021]
1
1.0
The number of mistakes made on epoch 0 is 0...
[-32775.52553273 -44754.43449578  11281.30601

...

[-35750.42025582 -93401.22780127 -49753.28138322 -49804.55984908
 -93230.18157634 -71550.43999346 -74725.88656205  30028.58791219
 -65399.76568033  27326.6201569 ]
7
7.0
The number of mistakes made on epoch 0 is 1...
[-37169.98462077 -74509.90675502 -90049.6748903  -69140.0852453
 -75738.56576745 -47061.29649799 -70471.63269075  -7246.25168955
 -64217.40122131  48373.01088965]
9
9.0
The number of mistakes made on epoch 0 is 1...
[ -25982.69393181  -93961.4582331   -49198.99830656  -43182.95702624
 -101764.14913279  -36342.31147174 -100167.26617302  -10538.27135791
  -77119.4462676    22803.99380008]
9
9.0
The number of mistakes made on epoch 0 is 1...
[ -46498.83409737  -84585.54262773  -57946.61096592  -57578.35342776
 -110199.37408786  -70536.84488038  -73653.13715716   46678.21492576
  -58092.23015945  -38429.18965094]
7
7.0
The number of mistakes made on epoch 0 is 1...
[-58525.43142466 -80857.75176628 -66473.82188762 -42722.60585183
 -96264.44360756 -81896.79711501 -84549.26802555

The number of mistakes made on epoch 0 is 1...
[-99402.0281947   55383.42517811 -59181.54333635 -69365.62137976
 -78494.06126563 -43690.58472856 -74211.73682755 -43880.84990769
 -56007.23145609 -68872.70035112]
1
1.0
The number of mistakes made on epoch 0 is 1...
[-55676.16875037 -60118.96372662 -81178.94712756 -49255.44820453
 -89833.85324276 -77934.21352543 -89865.05917427  17706.76048582
 -33369.70566217 -85137.94918728]
7
7.0
The number of mistakes made on epoch 0 is 1...
[-52911.87666363 -51935.62341573 -68817.99263405 -56161.32478504
 -54442.65425393  17599.86766689 -56425.742409   -61904.47607576
 -38143.38378551 -70440.84077706]
5
5.0
The number of mistakes made on epoch 0 is 1...
[-64745.79348062 -50726.06088008 -62518.68620929 -25591.89363379
 -82319.42097804  36803.95594812 -42102.93473736 -55823.77259131
 -77290.63531412 -64014.25321004]
5
5.0
The number of mistakes made on epoch 0 is 1...
[-93791.31317036  48831.03345453 -53952.34404382 -61954.40435173
 -73821.11547571 -41

The number of mistakes made on epoch 0 is 1...
[-86378.28005697  46660.05295129 -33063.82996327 -68534.90865186
 -38889.80072524 -46484.864556   -78348.14425699 -31841.58763246
 -72366.48513886 -90565.03753778]
1
1.0
The number of mistakes made on epoch 0 is 1...
[ -51377.50801208  -66271.88495373  -40891.0524247   -64052.64452303
  -45125.53443338  -52975.12711972   62404.53016121  -34462.77600944
  -64315.24218464 -107349.36842571]
6
6.0
The number of mistakes made on epoch 0 is 1...
[ 17011.66686042 -75691.35418342 -51259.62182773 -33594.46053924
 -76082.67143108 -34012.74660515 -58077.13780788 -42661.6909957
 -64232.39002919 -55336.64667958]
0
0.0
The number of mistakes made on epoch 0 is 1...
[ -59158.56415482  -94254.73382106  -61072.26453228  -48915.31314792
  -31110.00394169  -69716.15505752 -101531.73652348  -40411.07084634
  -52687.58025169   13581.56565369]
9
9.0
The number of mistakes made on epoch 0 is 1...
[ 34567.97889203 -40533.09054281 -20111.17270083 -48682.63808453
 

4.0
The number of mistakes made on epoch 0 is 1...
[-47193.36350527 -83912.18411183 -24433.61891349 -25649.41782394
 -68362.07775713 -51408.2705339  -27227.983383   -71612.74723746
  37079.59772999 -47413.17246757]
8
8.0
The number of mistakes made on epoch 0 is 1...
[ 41802.21398481 -43337.10451805 -33599.10213237 -49476.02800028
 -62921.66159084 -45817.06347807 -53271.44384554 -46893.08940455
 -31688.60014201 -43606.76336416]
0
0.0
The number of mistakes made on epoch 0 is 1...
[-48229.99430883 -63597.91326811 -43539.0144695  -17232.47012705
 -37767.1335423  -48057.75695621 -42304.23591863 -64894.89708546
  23775.92318539 -39672.45678932]
8
8.0
The number of mistakes made on epoch 0 is 1...
[-44844.1015262  -77793.75205888 -38489.55315196 -49581.33518087
  12366.72860173 -31361.75489079 -52699.99043176 -28385.1057225
 -49377.80251174 -26702.47050183]
4
4.0
The number of mistakes made on epoch 0 is 1...
[-43227.20076215 -71680.05378492    214.93931184 -54363.10194595
  17508.90936714 

KeyboardInterrupt: 

In [ ]:
# Make plots of losses to see whether it is converging
# Shapes of all matrices
# Take a look at the actual data in each matrix
# Take a look at each weight
# Take a look at whether kernel arguments are being sent in correctly
# Time run